<a href="https://colab.research.google.com/github/jpmucciolo/Curso-Ocean-19-03---Ciencia-de-Dados/blob/main/Curso_Ocean_19_03_ciencia_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>